# Teoria Secular Linear

Aplicando a Teoria Secular Linear a um conjundo genérico de planetas

In [171]:
# Importando módulos
import numpy as np
import pandas as pd
from scipy.integrate import quad
import os

## Constantes
A constante da gravitação universal $G$ tem o valor para o qual $M_{\odot} = 1$.

In [172]:
# Constante da gravitação universal
G = (0.01720209895)**2 # para o SS AU^3 d^-2 M_sol^-1

## Massa do Sol

In [173]:
# Massa do Sol
M = 1 # Massa unitária para o Sol

## Dados dos planetas

In [174]:
# Leitura dos dados
pl = pd.read_excel('planetas.xls')

# Semieixo maior
a = pl['a'] # AU

# Comprimento do vetor de semieixos.
len_a = len(a)

# Dados das massas
m = pl['m'] # Considerando massa unitária para o Sol

# Dados dos movimentos médios n
n = (G * (M + m)/a**3)**(1/2) # rad/day

# Excentricidade
e = pl['e']

# Inclinação
inc = pl['inc']

# Longitude do periastro
varpi = pl['varpi']

# Nodo ascendente
capom = pl['capom']


### Verificando dados de entrada

In [175]:
pl

,Planeta,m,a,e,inc,varpi,capom
0,Jupiter,0.000955,5.202545,0.047462,1.30667,13.983865,100.0381
1,Saturno,0.000286,9.554841,0.057548,2.48795,88.719425,113.1334


## Definição das funções

### Função $\alpha$

$$\alpha_{i, j} = \frac{\text{min}\{a_i, a_j\}}{\text{max}\{a_i, a_j\}}$$

In [176]:
# Função alpha
def al(x, y):
    return min(x, y) / max(x, y)

### Coeficientes de Laplace $b_{\psi}^{k}(\alpha_{i,j})$

$$b_{\psi}^{\left(k\right)}\left(\xi\right)=\frac{1}{\pi}\int_{0}^{2\pi}\left[1-2\xi\cos\theta+\xi^{2}\right]^{-\psi}\cos\left(k\theta\right)\,d\theta$$

Para nossos propósitos, $\psi = 3/2$. O valor de $k$ será igual a $1$ ou $2$.

In [177]:
# Função para coeficientes de Laplace
def coeff_laplace(k, al):
    f = lambda x: (1 - 2 * al * np.cos(x) + al**2)**(-3/2) * np.cos(k * x)
    coeff = quad(f, 0, 2*np.pi)
    return (1.0 / np.pi) * coeff[0]

### Resolvendo as equações diferenciais linearizadas

Para a resolução das equações,

\begin{align}
    \frac{dk_i}{dt} &=&-&\sum_{j=1}^N A_{ij} h_j \\
    \frac{dh_i}{dt} &=& &\sum_{j=1}^N A_{ij} k_j \\
    \frac{dq_i}{dt} &=&-&\sum_{j=1}^N B_{ij} p_j \\
    \frac{dp_i}{dt} &=& &\sum_{j=1}^N B_{ij} h_j 
\end{align}

onde os termos $k, h, p, q$ são as variáveis regularizadas.

As matrizes $A$ e $B$ são dadas por,

\begin{align}
    A_{ij}&=&-&\frac{G}{4n_{i}a_{i}^{2}}\frac{m_{j}\alpha_{ij}}{a_{ij}} b_{3/2}^{\left(2\right)}\left(\alpha_{ij}\right)&(i\ne j)\\
    A_{ii}&=& & \frac{G}{4n_{i}a_{i}^{2}}\sum_{\substack{j=1\\j\ne i}}^{N}\frac{m_{j}\alpha_{ij}}{a_{ij}} b_{3/2}^{\left(1\right)}\left(\alpha_{ij}\right)&\\
    B_{ij}&=& &\frac{G}{4n_{i}a_{i}^{2}}\frac{m_{j}\alpha_{ij}}{a_{ij}} b_{3/2}^{\left(1\right)}\left(\alpha_{ij}\right)&(i\ne j)\\
    B_{ii}&=& - &\frac{G}{4n_{i}a_{i}^{2}}\sum_{\substack{j=1\\j\ne i}}^{N}\frac{m_{j}\alpha_{ij}}{a_{ij}} b_{3/2}^{\left(1\right)}\left(\alpha_{ij}\right)&=-A_{ij}
\end{align}

Para efetuar o calculo, foram criadas as funções com as respectivas partes

\begin{align}
    \text{parte1} & = & \frac{G}{4n_{i}a_{i}^{2}}\\
    \text{parte2} & = & \frac{m_{j}\alpha_{ij}}{a_{ij}}\\
    \text{parte2} & = & b_{3/2}^{\left(k\right)}\left(\alpha_{ij}\right)
\end{align}

In [178]:
# Funções para o cálculo das matrizes divida em partes
def parte1(i):
    return G/(4 * n[i] * a[i]**2)

def parte2(i, j):
    return (m[j] * al(a[i], a[j])) / max(a[i], a[j])

def parte3(k, i, j):
    return coeff_laplace(k, al(a[i], a[j]))

## Matriz A

In [179]:
# Matriz A
A = np.zeros((len_a, len_a))
for i in range(0, len_a):
    for j in range(0,len_a):
        if j != i:
            A[i, j] = - parte1(i) * parte2(i, j) * parte3(2, i, j)
        else:
            p = 0
            for k in range(0, len_a):
                if k != i:
                    p = p + parte2(i, k) * parte3(1, i, k)
            A[i, j] = parte1(i) * p


# Convertendo a matriz A para unidades de arcsec / yr
#A = A * (180/np.pi) * 365.25 * 3600

# Convertendo a matriz A para unidades de deg / yr
A = A * (180/np.pi) * 365.25

# Obtendo a matriz simétrica A
A_cal = np.zeros((len_a,len_a))
for i in range(0,len_a):
    for j in range(0,len_a):
        if i == j:
            A_cal[i, i] = A[i,i]
        else:
            A_cal[i, j] = ((a[i] * np.sqrt(m[i] * n[i])) \
                           / (a[j] * np.sqrt(m[j] * n[j]))) * A[i, j]

# Obtendo os autovetores e autovalres de A
A_eigen = np.linalg.eig(A_cal)

## Matriz B

In [180]:
# Matriz B

B = np.zeros((len_a, len_a))
for i in range(0, len_a):
    for j in range(0,len_a):
        if j != i:
            B[i, j] =  parte1(i) * parte2(i, j) * parte3(1, i, j)
        else:
            B[i, i] = -A[i, i]


# Convertendo a matriz B para unidades de arcsec / yr
#B = B * (180/np.pi) * 365.25 * 3600

# Convertendo a matriz B para unidades de deg / yr
B = B * (180/np.pi) * 365.25

# Obtendo a matriz simétrica B
B_cal = np.zeros((len_a,len_a))
for i in range(0,len_a):
    for j in range(0,len_a):
        if i == j:
            B_cal[i, i] = -A[i,i]
        else:
            B_cal[i, j] = ((a[i] * np.sqrt(m[i] * n[i])) \
                           / (a[j] * np.sqrt(m[j] * n[j]))) * B[i, j]

# Obtendo os autovetores e autovalres de B
B_eigen = np.linalg.eig(B_cal)

### Matrizes A e B e seus respectivos autovetores e autovalores

In [181]:
# Resultados

#print('Resultados em arcsec/yr')
#print('--------------------\n')

print('Resultados em deg/yr')
print('--------------------\n')

print('\n Matriz simétrica A')
print(A_cal)

print('\n Autovetores de A')
print(A_eigen[1])

print('\n Autovalores de A')
print(A_eigen[0])

print('\n Matriz simétrica B')
print(B_cal)

print('\n Autovetores de B')
print(B_eigen[1])

print('\n Autovalores de B')
print(B_eigen[0])

Resultados em deg/yr
--------------------


 Matriz simétrica A

[[ 0.00203832 -0.00208916]
 [-0.00208916  0.00502575]]

 Autovetores de A
[[-0.88927338  0.45737605]
 [-0.45737605 -0.88927338]]

 Autovalores de A
[ 0.00096381  0.00610026]

 Matriz simétrica B
[[-0.00203832  0.00320064]
 [ 0.00320064 -0.00502575]]

 Autovetores de B
[[ 0.84347654 -0.53716602]
 [ 0.53716602  0.84347654]]

 Autovalores de B
[  8.67361738e-19  -7.06406493e-03]


## Condições de contorno

### Cálculo das variáveis regulares

\begin{equation}
    \begin{array}{cc}
        h_j = e_j\sin(\varpi_j) & k_j = e_j\cos(\varpi_j) \\
        p_j = I_j\sin(\Omega_j) & q_j = I_j\cos(\Omega_j)
    \end{array}
\end{equation}

In [182]:
h =  k = p = q = np.zeros(len_a)

for j in range(0,len_a):
    h[j] = e[j] * np.sin(varpi[j])
    k[j] = e[j] * np.cos(varpi[j])
    p[j] = inc[j] * np.sin(capom[j])
    q[j] = inc[j] * np.cos(capom[j])

In [183]:
#omega1 = np.pi * omega1 / 180
h

array([ 1.15115136,  2.4863322 ])